In [2]:
!pip install langchain openai tiktoken faiss-cpu python-dotenv
!pip install pypdf unstructured
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 51.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.4/181.4 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 9.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [8]:
!pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 30.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [16]:
# Install necessary packages
!pip install langchain langchain_community langchain_openai faiss-cpu tiktoken requests beautifulsoup4 tenacity

# Import required libraries
import os
import re
import time
import random
import requests
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt, wait_exponential
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

# Define function to fetch and parse SEC filings with retry logic
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=60))
def fetch_sec_filing(url):
    # Rotate user agents to avoid detection
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0'
    ]

    headers = {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0'
    }

    # Add random delay to avoid rate limiting
    time.sleep(random.uniform(1, 3))

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch the URL: {url}, Status code: {response.status_code}")

    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract text content, removing script and style elements
    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()

    # Clean up text
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

# Function to decompose a query into sub-queries
def decompose_query(query):
    llm = ChatOpenAI(model="gpt-4", temperature=0.3)

    decomposition_prompt = f"""
    Break down the 'Question' into multiple sub-queries that will help retrieve specific information about Tesla and GM from their SEC filings.

    Guidelines:
    1. The set of sub-queries together should capture the complete information needed to answer the question.
    2. Each sub-query should ask for just one piece of information about one specific company.
    3. For each sub-query, only mention the information you're trying to get. Don't use verbs like "retrieve" or "find".
    4. Include the company name mentioned in each sub-query.
    5. Make the sub-queries specific enough to match sections in SEC filings about manufacturing processes, production facilities, and safety standards.
    6. Focus on terms likely to appear in formal financial documents.

    Question: {query}

    Return only the sub-queries, each enclosed in angle brackets like <sub-query>. Do not include any explanations.
    """

    response = llm.invoke(decomposition_prompt)

    # Extract sub-queries from the response
    sub_queries = re.findall(r'<(.*?)>', response.content)

    return sub_queries

# URLs for the 10-K SEC filings
tesla_url = "https://www.sec.gov/Archives/edgar/data/1318605/000162828024002390/tsla-20231231.htm"
gm_url = "https://www.sec.gov/Archives/edgar/data/1467858/000146785824000031/gm-20231231.htm"

# Main function to implement RAG with query decomposition
def rag_with_decomposition(query):
    # Step 1: Fetch and process SEC filings
    try:
        print("Fetching Tesla 10-K filing...")
        tesla_text = fetch_sec_filing(tesla_url)
        print(f"Tesla 10-K length: {len(tesla_text)} characters")

        print("Fetching GM 10-K filing...")
        gm_text = fetch_sec_filing(gm_url)
        print(f"GM 10-K length: {len(gm_text)} characters")
    except Exception as e:
        print(f"Error fetching SEC filings: {e}")
        return f"Failed to access SEC filings: {str(e)}"

    # Create Document objects
    tesla_doc = Document(page_content=tesla_text, metadata={"source": "Tesla 10-K", "company": "Tesla"})
    gm_doc = Document(page_content=gm_text, metadata={"source": "GM 10-K", "company": "General Motors"})

    # Combine documents
    all_docs = [tesla_doc, gm_doc]

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )

    all_splits = text_splitter.split_documents(all_docs)
    print(f"Split {len(all_docs)} documents into {len(all_splits)} chunks")

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(all_splits, embeddings)

    # Step 2: Decompose the query into sub-queries
    sub_queries = decompose_query(query)
    print(f"Decomposed into {len(sub_queries)} sub-queries:")
    for i, sq in enumerate(sub_queries):
        print(f"{i+1}. {sq}")

    # Step 3: Retrieve relevant documents for each sub-query
    all_relevant_docs = []
    sub_query_results = {}

    for i, sub_query in enumerate(sub_queries):
        # Extract company name from sub-query to filter results
        company_filter = None
        if "tesla" in sub_query.lower():
            company_filter = "Tesla"
        elif "gm" in sub_query.lower() or "general motors" in sub_query.lower():
            company_filter = "General Motors"

        # Retrieve documents with metadata filtering when applicable
        if company_filter:
            try:
                docs = vectorstore.similarity_search(
                    sub_query,
                    k=4,  # Retrieve more documents per sub-query
                    filter={"company": company_filter}
                )
            except Exception as e:
                print(f"Error with filtered search: {e}")
                docs = vectorstore.similarity_search(sub_query, k=4)
        else:
            docs = vectorstore.similarity_search(sub_query, k=4)

        # Store results by sub-query
        sub_query_results[sub_query] = docs
        all_relevant_docs.extend(docs)

    # Step 4: Remove duplicates
    unique_docs = []
    unique_content = set()
    for doc in all_relevant_docs:
        if doc.page_content not in unique_content:
            unique_content.add(doc.page_content)
            unique_docs.append(doc)

    # Step 5: Create a structured context for the final answer
    structured_context = ""
    for i, (sub_query, docs) in enumerate(sub_query_results.items()):
        structured_context += f"\nInformation for: {sub_query}\n"
        for j, doc in enumerate(docs):
            structured_context += f"Document {j+1} (from {doc.metadata.get('company', 'unknown')}):\n{doc.page_content}\n\n"

    # Step 6: Generate the final answer
    llm = ChatOpenAI(model="gpt-4", temperature=0.5)

    final_prompt = f"""
    You are an expert financial analyst comparing Tesla and General Motors based on their SEC filings.

    TASK: Create a comprehensive comparison of Tesla and GM focusing on:
    1. Their approaches to manufacturing and production for electric vehicles
    2. Their production locations worldwide
    3. The safety standards followed in their vehicles

    Below is information extracted from their recent SEC filings, organized by sub-query:
    {structured_context}

    Original Question: {query}

    IMPORTANT INSTRUCTIONS:
    - If the provided information is insufficient for any aspect, acknowledge this limitation but still provide analysis based on what IS available
    - Use specific details from the documents whenever possible
    - Structure your answer with clear sections for each aspect of the comparison
    - For manufacturing approaches, focus on vertical integration vs. supplier relationships
    - For production locations, list specific facilities and their purposes
    - For safety standards, mention specific ratings, technologies, and compliance measures

    Format your response with these sections:
    1. Manufacturing and Production Approaches
    2. Production Locations
    3. Safety Standards
    """

    response = llm.invoke(final_prompt)
    return response.content

# Alternative implementation using LangChain's newer chain structure
def rag_with_decomposition_langchain_style(query):
    # Step 1: Fetch and process SEC filings
    try:
        print("Fetching Tesla 10-K filing...")
        tesla_text = fetch_sec_filing(tesla_url)
        print(f"Tesla 10-K length: {len(tesla_text)} characters")

        print("Fetching GM 10-K filing...")
        gm_text = fetch_sec_filing(gm_url)
        print(f"GM 10-K length: {len(gm_text)} characters")
    except Exception as e:
        print(f"Error fetching SEC filings: {e}")
        return f"Failed to access SEC filings: {str(e)}"

    # Create Document objects
    tesla_doc = Document(page_content=tesla_text, metadata={"source": "Tesla 10-K", "company": "Tesla"})
    gm_doc = Document(page_content=gm_text, metadata={"source": "GM 10-K", "company": "General Motors"})

    # Combine documents
    all_docs = [tesla_doc, gm_doc]

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )

    all_splits = text_splitter.split_documents(all_docs)
    print(f"Split {len(all_docs)} documents into {len(all_splits)} chunks")

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(all_splits, embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

    # Step 2: Decompose the query
    sub_queries = decompose_query(query)
    print(f"Decomposed into {len(sub_queries)} sub-queries:")
    for i, sq in enumerate(sub_queries):
        print(f"{i+1}. {sq}")

    # Step 3: Create a retrieval chain for each sub-query and combine results
    all_docs = []
    for sub_query in sub_queries:
        docs = retriever.invoke(sub_query)
        all_docs.extend(docs)

    # Remove duplicates
    unique_docs = []
    unique_content = set()
    for doc in all_docs:
        if doc.page_content not in unique_content:
            unique_content.add(doc.page_content)
            unique_docs.append(doc)

    # Create context from unique documents
    context = "\n\n".join([doc.page_content for doc in unique_docs])

    # Step 4: Create the RAG prompt template
    prompt = ChatPromptTemplate.from_template("""
    You are an expert financial analyst comparing Tesla and General Motors based on their SEC filings.

    TASK: Create a comprehensive comparison of Tesla and GM focusing on:
    1. Their approaches to manufacturing and production for electric vehicles
    2. Their production locations worldwide
    3. The safety standards followed in their vehicles

    Below is information extracted from their recent SEC filings:
    {context}

    Original Question: {question}

    IMPORTANT INSTRUCTIONS:
    - If the provided information is insufficient for any aspect, acknowledge this limitation but still provide analysis based on what IS available
    - Use specific details from the documents whenever possible
    - Structure your answer with clear sections for each aspect of the comparison
    - For manufacturing approaches, focus on vertical integration vs. supplier relationships
    - For production locations, list specific facilities and their purposes
    - For safety standards, mention specific ratings, technologies, and compliance measures

    Format your response with these sections:
    1. Manufacturing and Production Approaches
    2. Production Locations
    3. Safety Standards
    """)

    # Step 5: Create the LLM
    llm = ChatOpenAI(model="gpt-4", temperature=0.5)

    # Step 6: Create and run the chain
    chain = (
        {"context": lambda x: context, "question": lambda x: x}
        | prompt
        | llm
        | StrOutputParser()
    )

    response = chain.invoke(query)
    return response

# Main execution
if __name__ == "__main__":
    # Main question from the assignment
    main_question = "How do Tesla and GM's approaches to manufacturing and production compare, particularly for electric vehicles? Where are their vehicles produced? What are the safety standards followed in their vehicles?"

    # Run the RAG pipeline with query decomposition
    print("Running RAG with query decomposition...")
    answer = rag_with_decomposition(main_question)
    print("\nFinal Answer:")
    print(answer)

    # Alternatively, you can use the LangChain style implementation
    # answer = rag_with_decomposition_langchain_style(main_question)
    # print("\nFinal Answer (LangChain style):")
    # print(answer)


Running RAG with query decomposition...
Fetching Tesla 10-K filing...
Tesla 10-K length: 422470 characters
Fetching GM 10-K filing...
GM 10-K length: 487665 characters
Split 2 documents into 1191 chunks
Decomposed into 6 sub-queries:
1. What is Tesla's approach to manufacturing and production for electric vehicles?
2. What is GM's approach to manufacturing and production for electric vehicles?
3. Where are Tesla's electric vehicles produced?
4. Where are GM's electric vehicles produced?
5. What safety standards does Tesla follow in their vehicles?
6. What safety standards does GM follow in their vehicles?

Final Answer:
1. Manufacturing and Production Approaches

Tesla's approach to manufacturing and production for electric vehicles is focused on growing its manufacturing capacity, including the capacity for manufacturing new vehicle models such as the Cybertruck and next-generation platform. Tesla aims to ramp all its production vehicles to their installed production capacities while 